### Reference about the paper `Robot Sensor Calibration: Solving AX = XB on the Euclidean Group`
### Example python code of upper paper.

In [6]:
import numpy as np
import scipy

A1 = np.array([[-0.989992,    -0.141120,  0.000,  0.0],
                [0.141120,      -0.989992,  0.000,  0.0],
                [0.000000,      0.0000000,  1.000,    0],
                [0       ,              0,      0,    1]])

B1 = np.array([[-0.989992, -0.138307, 0.028036, -26.9559],
                 [0.138307 , -0.911449, 0.387470, -96.1332],
                 [-0.028036 ,  0.387470, 0.921456, 19.4872],
                 [0        ,        0,     0, 1]])

A2 = np.array([[0.07073, 0.000000, 0.997495, -400.000],
                [0.000000, 1.000000, 0.000000, 0.000000],
                [-0.997495, 0.000000, 0.070737, 400.000],
                [0, 0, 0,1]])

B2 = np.array([[ 0.070737, 0.198172, 0.997612, -309.543],
                [-0.198172, 0.963323, -0.180936, 59.0244],
                [-0.977612, -0.180936, 0.107415, 291.177],
                [0, 0, 0, 1]])

### Convert SO(3) to Lie Group Theta

In [7]:
def log_group_skew(T):
    """
        Convert to Log matrix form. Return skew matrix form.
        This is code of Lemma 2.
    """
    R = T[0:3, 0:3] # Rotation matrix
    # Lemma 2
    theta = np.arccos((np.trace(R) - 1)/2)

    logr = np.array([R[2,1] - R[1,2], R[0,2] - R[2,0], R[1,0] - R[0,1]]) * theta / (2*np.sin(theta))
    return logr # return skew matrix form

def log_group_matrix(T):
    """
        Convert to Log matrix form. Return 3x3 matrix form.
        This is code of Lemma 2.
    """
    R = T[0:3, 0:3] # Rotation matrix
    # Lemma 2
    theta = np.arccos((np.trace(R) - 1)/2)

    theta_ = R - R.T
    log_r = theta_ * theta / (2*np.sin(theta))
    return log_r

A1_log = log_group_skew(A1)
A2_log = log_group_skew(A2)
B1_log = log_group_skew(B1)
B2_log = log_group_skew(B2)

A1_log_matrix = log_group_matrix(A1)
A2_log_matrix = log_group_matrix(A2)
B1_log_matrix = log_group_matrix(B1)
B2_log_matrix = log_group_matrix(B2)

In [3]:
print(A1_log)
print(A2_log)
print(B1_log)
print(B2_log)
print()
print(A1_log_matrix)
print(A2_log_matrix)
print(B1_log_matrix)
print(B2_log_matrix)

[0.         0.         2.99992225]
[0.         1.50000334 0.        ]
[0.         0.59600347 2.94020018]
[ 0.          1.48513803 -0.29800445]

[[ 0.         -2.99992225  0.        ]
 [ 2.99992225  0.          0.        ]
 [ 0.          0.          0.        ]]
[[ 0.          0.          1.50000334]
 [ 0.          0.          0.        ]
 [-1.50000334  0.          0.        ]]
[[ 0.         -2.94020018  0.59600347]
 [ 2.94020018  0.          0.        ]
 [-0.59600347  0.          0.        ]]
[[ 0.          0.29800445  1.48513803]
 [-0.29800445  0.          0.        ]
 [-1.48513803  0.          0.        ]]


### Calibrate two equations

In [8]:
def calibrate(A, B):
    """
        Solve AX=XB calibration.
    """

    data_num = len(A)
    
    M = np.zeros((3,3))
    C = np.zeros((3*data_num, 3))
    d = np.zeros((3*data_num, 1))

    # columns of matrix A, B
    alpha = log_group_skew(A[0])     # 3 x 1
    beta = log_group_skew(B[0])      # 3 x 1
    alpha_2 = log_group_skew(A[1]) # 3 x 1
    beta_2 = log_group_skew(B[1])  # 3 x 1
    alpha_3 = np.cross(alpha, alpha_2)  # 3 x 1
    beta_3 = np.cross(beta, beta_2)     # 3 x 1

    # print(alpha)
    # print(alpha_2)
    # print(beta)
    # print(beta_2)
    # assert np.array_equal(np.cross(alpha, alpha_2), np.zeros(3)) 
    # assert np.array_equal(np.cross(beta, beta_2), np.zeros(3))

    # M = \Sigma (beta * alpha.T)
    M1 = np.dot(beta.reshape(3,1),alpha.reshape(3,1).T)
    M2 = np.dot(beta_2.reshape(3,1),alpha_2.reshape(3,1).T)
    M3 = np.dot(beta_3.reshape(3,1),alpha_3.reshape(3,1).T)
    M = M1+M2+M3

    # theta_x = (M.T * M)^(-1/2) * M.T    
    # theta_x = np.dot(np.sqrt(np.linalg.inv((np.dot(M.T, M)))), M.T)
    # RuntimeWarning: invalid value encountered in sqrt: np.sqrt results nan values
    theta_x = np.dot(scipy.linalg.sqrtm(np.linalg.inv((np.dot(M.T, M)))), M.T)  # rotational info

    # A_ = np.array([alpha, alpha_2, alpha_3])
    # B_ = np.array([beta, beta_2, beta_3])
    # B_inv = np.linalg.inv(B_)
    # theta_x = A * B_inv

    for i in range(data_num):
        A_rot   = A[i][0:3,0:3]
        A_trans = A[i][0:3, 3]
        B_rot   = B[i][0:3,0:3]
        B_trans = B[i][0:3, 3]
        
        C[3*i:3*i+3, :] = np.eye(3) - A_rot
        d[3*i:3*i+3, 0] = A_trans - np.dot(theta_x, B_trans)


    b_x = np.dot(np.linalg.inv(np.dot(C.T, C)), np.dot(C.T, d))     # translational info

    return theta_x, b_x

In [43]:
A = [A1, A2]
B = [B1, B2]

theta_x, b_x = calibrate(A, B)
cali_ = np.append(theta_x, b_x, axis=1)
cali_

array([[ 1.        ,  0.        ,  0.        ,  9.99989033],
       [ 0.        ,  0.98014571, -0.19827854, 50.00001805],
       [ 0.        ,  0.19827854,  0.98014571, 99.99990214]])

# Load the data

In [9]:
import numpy as np
from util import *

A = np.load("./data/A_matrix-3.npy")
B = np.load("./data/B_matrix-3.npy")

print(len(A))

10


In [11]:
cali_list = []

for i in range(len(A)-1):
    A_list = [A[i], A[i+1]]
    B_list = [B[i], B[i+1]]

    theta_x, b_x = calibrate(A_list, B_list)
    print(np.degrees(r2rpy(theta_x)))
    # print(b_x.ravel())
    cali_ = np.append(theta_x, b_x, axis=1)
    cali_list.append(cali_)


[ -72.53986096   11.84439376 -135.33739758]
[ -46.03299202   12.20338428 -135.48483795]
[ -93.93426915   12.1705647  -135.0752916 ]
[  68.9414419    13.86967172 -134.9473419 ]
[  65.49051498   14.26110322 -134.97554802]
[ -88.50752629   13.92603322 -135.01285659]
[  57.62896652   15.38502297 -135.05898894]
[ -91.68360605   15.07579257 -134.97029756]
[  77.30012675   16.6177148  -134.92315774]


In [12]:
theta_x, b_x = calibrate(A, B)


In [13]:
print(np.degrees(r2rpy(theta_x)))
print(b_x.ravel())

[ -72.53986096   11.84439376 -135.33739758]
[0.29324188 0.12932087 0.12916188]
